# Export PyTorch Model to ONNX
### Mục tiêu
- Chạy mô hình PyTorch và xuất sang ONNX 
- Kiểm tra tính hợp lệ của mô hình ONNX
- Chạy inference với ONNX Runtime
- So sánh kết quả với PyTorch

### Bước 1: Cài đặt môi trường

In [15]:
!pip install torch torchvision torchaudio onnx onnxscript onnxruntime

### Bước 2: Kiểm tra phiên bản

In [16]:
import torch
import onnx
import onnxruntime
print(f'PyTorch Version: {torch.__version__}')
print(f'ONNX Version: {onnx.__version__}')
print(f'ONNX Runtime Version: {onnxruntime.__version__}')

PyTorch Version: 2.6.0
ONNX Version: 1.17.0
ONNX Runtime Version: 1.20.1


### Bước 3: Định nghĩa mô hình PyTorch

In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Khởi tạo mô hình
torch_model = MyModel()
print('Mô hình đã được khởi tạo!')

Mô hình đã được khởi tạo!


### Bước 4: Xuất mô hình sang ONNX

In [18]:
torch_input = torch.randn(1, 1, 32, 32)
onnx_program = torch.onnx.dynamo_export(torch_model, torch_input)
onnx_program.save('my_image_classifier.onnx')
print('Mô hình đã xuất thành công sang ONNX!')

/opt/anaconda3/envs/env_onnx/lib/python3.10/site-packages/torch/onnx/_internal/_exporter_legacy.py:101: UserWarning: torch.onnx.dynamo_export only implements opset version 18 for now. If you need to use a different opset version, please register them with register_custom_op.
  warnings.warn(


Mô hình đã xuất thành công sang ONNX!


### Bước 5: Kiểm tra tính hợp lệ của mô hình ONNX

In [19]:
onnx_model = onnx.load('my_image_classifier.onnx')
onnx.checker.check_model(onnx_model)
print('Mô hình ONNX hợp lệ!')

Mô hình ONNX hợp lệ!


### Bước 6: Chạy inference với ONNX Runtime

In [20]:
ort_session = onnxruntime.InferenceSession('my_image_classifier.onnx', providers=['CPUExecutionProvider'])

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

onnxruntime_input = {k.name: to_numpy(v) for k, v in zip(ort_session.get_inputs(), [torch_input])}
onnxruntime_outputs = ort_session.run(None, onnxruntime_input)[0]
print('Dự đoán từ ONNX Runtime:', onnxruntime_outputs)

Dự đoán từ ONNX Runtime: [[-0.04551338 -0.01886104 -0.07032115 -0.08774921  0.09030626 -0.01017094
  -0.11590829  0.03748711  0.06311531  0.09808487]]


### Bước 7: So sánh kết quả giữa PyTorch và ONNX

In [21]:
torch_outputs = torch_model(torch_input)
assert len(torch_outputs) == len(onnxruntime_outputs)
for torch_output, onnxruntime_output in zip(torch_outputs, onnxruntime_outputs):
    torch.testing.assert_close(torch_output, torch.tensor(onnxruntime_output))
print('Kết quả của PyTorch và ONNX Runtime khớp nhau!')

Kết quả của PyTorch và ONNX Runtime khớp nhau!


## Kết luận
- Mô hình xuất sang ONNX thành công
- Chạy inference trên ONNX Runtime thành công
- Kết quả khớp giữa PyTorch và ONNX